In [5]:
import asyncio
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import aiohttp


nest_asyncio.apply()  # Allow running nested event loops in Jupyter Notebook

async def fetch_page(session, url):
    async with session.get(url) as response:
        return await response.text()

async def click_show_buttons(driver, num_buttons):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'show-button')))
        show_buttons = driver.find_elements(By.CLASS_NAME, 'show-button')

        for i in range(min(num_buttons, len(show_buttons))):
            if show_buttons[i].is_displayed() and show_buttons[i].is_enabled():
                try:
                    show_buttons[i].click()
                except Exception as ex:
                    # Handle any exception during click, such as ElementClickInterceptedException
                    print(f"Exception occurred: {ex}")
                time.sleep(1)  # Add a short delay to allow the content to load (optional)
    except TimeoutException:
        # Handle the TimeoutException if the 'show-button' element is not found
        pass

    
async def extract_phone_numbers(driver):
    phone_numbers = []

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'phone-item')))
        phone_items = driver.find_elements(By.CLASS_NAME, 'phone-item')

        for phone_item in phone_items:
            phone_number = phone_item.find_element(By.TAG_NAME, 'span').text.strip()
            phone_numbers.append(phone_number)

    except TimeoutException:
        # Handle the TimeoutException if the 'phone-item' element is not found
        pass

    return phone_numbers

async def process_link(session, link):
    href = link["href"]
    url = f'https://lalafo.az{href}'
    html_content = await fetch_page(session, url)

    soup = BeautifulSoup(html_content, "html.parser")

    num_buttons_to_click = 4
    click_show_buttons(driver, num_buttons_to_click)

    extracted_phone_numbers = extract_phone_numbers(driver)
    return extracted_phone_numbers

sync def main_async(all_links):
    loop = asyncio.get_event_loop()

    async with aiohttp.ClientSession() as session:
        tasks = [process_link(session, link) for link in all_links]
        phone_numbers_list = await asyncio.gather(*tasks)

    # Create a DataFrame and save to CSV
    df = pd.DataFrame(phone_numbers_list, columns=["Phone Numbers"])
    df.to_csv("phone_numbers.csv", index=False)

def main(all_links):
    # Initialize Chrome WebDriver
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome WebDriver in headless mode
    driver = webdriver.Chrome(options=chrome_options)

    for link in all_links:
        href = link["href"]
        url = f'https://lalafo.az{href}'
        driver.get(url)
        time.sleep(2)

        # Click on the first four "show" buttons
        num_buttons_to_click = 4
        click_show_buttons(driver, num_buttons_to_click)

        extracted_phone_numbers = extract_phone_numbers(driver)
        print(extracted_phone_numbers)

    driver.quit()

if __name__ == "__main__":
    # Get the links using BeautifulSoup
    url = "https://lalafo.az/azerbaijan/nedvizhimost"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    all_links = soup.find_all("a", href=True)

    asyncio.run(main_async(all_links))
    main(all_links)

SyntaxError: invalid syntax (1253163622.py, line 69)